# ടെക്സ്റ്റ് വർഗ്ഗീകരണ പ്രവർത്തി

നാം പറഞ്ഞതുപോലെ, നാം **AG_NEWS** ഡാറ്റാസെറ്റിനെ അടിസ്ഥാനമാക്കി ലളിതമായ ടെക്സ്റ്റ് വർഗ്ഗീകരണ പ്രവർത്തനത്തിൽ ശ്രദ്ധ കേന്ദ്രീകരിക്കും, ഇത് വാർത്താ തലക്കെട്ടുകൾ നാല് വിഭാഗങ്ങളിൽ ഒന്നായി വർഗ്ഗീകരിക്കുകയാണ്: ലോകം, കായികം, ബിസിനസ്സ്, സയൻസ്/ടെക്.

## ഡാറ്റാസെറ്റ്

ഈ ഡാറ്റാസെറ്റ് [`torchtext`](https://github.com/pytorch/text) മോഡ്യൂളിൽ ഉൾപ്പെടുത്തിയിട്ടുള്ളതാണ്, അതിനാൽ നാം എളുപ്പത്തിൽ അതിലേക്ക് പ്രവേശിക്കാം.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

ഇവിടെ, `train_dataset` ഉം `test_dataset` ഉം ലേബൽ (ക്ലാസിന്റെ സംഖ്യ)യും ടെക്സ്റ്റും യോജിപ്പായി നൽകുന്ന ശേഖരങ്ങളാണ്, ഉദാഹരണത്തിന്:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

അപ്പോൾ, നമ്മുടെ ഡാറ്റാസെറ്റിൽ നിന്നുള്ള ആദ്യ 10 പുതിയ തലക്കെട്ടുകൾ പ്രിന്റ് ചെയ്യാം:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

ഡാറ്റാസെറ്റുകൾ ഇറ്ററേറ്ററുകളായതിനാൽ, ഡാറ്റ പലതവണ ഉപയോഗിക്കണമെങ്കിൽ അത് ലിസ്റ്റായി മാറ്റേണ്ടതുണ്ട്:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## ടോക്കനൈസേഷൻ

ഇപ്പോൾ നമുക്ക് ടെക്സ്റ്റ് **സംഖ്യകളായി** മാറ്റേണ്ടതുണ്ട്, അവ ടെൻസറുകളായി പ്രതിനിധീകരിക്കാനാകും. വാക്ക്-നിലവാരത്തിലുള്ള പ്രതിനിധാനം വേണമെങ്കിൽ, നമുക്ക് രണ്ട് കാര്യങ്ങൾ ചെയ്യണം:
* ടെക്സ്റ്റ് **ടോക്കണുകളായി** വിഭജിക്കാൻ **ടോക്കനൈസർ** ഉപയോഗിക്കുക
* ആ ടോക്കണുകളുടെ **വോകാബുലറി** നിർമ്മിക്കുക.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

വാക്ക് സമ്പത്തിനെ ഉപയോഗിച്ച്, നാം എളുപ്പത്തിൽ ടോക്കൺ ചെയ്ത സ്ട്രിംഗ് ഒരു സംഖ്യാ സമാഹാരമായി എൻകോഡ് ചെയ്യാൻ കഴിയും:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## വാക്കുകളുടെ ബാഗ് ടെക്സ്റ്റ് പ്രതിനിധാനം

വാക്കുകൾ അർത്ഥം പ്രതിനിധാനം ചെയ്യുന്നതിനാൽ, വാചകത്തിലെ ക്രമം നോക്കാതെ ചിലപ്പോൾ ഓരോ വാക്കുകളും നോക്കി ടെക്സ്റ്റിന്റെ അർത്ഥം മനസിലാക്കാൻ കഴിയും. ഉദാഹരണത്തിന്, വാർത്തകൾ വർഗ്ഗീകരിക്കുമ്പോൾ *weather*, *snow* പോലുള്ള വാക്കുകൾ *weather forecast* സൂചിപ്പിക്കാം, അതേസമയം *stocks*, *dollar* പോലുള്ള വാക്കുകൾ *financial news* എന്ന വിഭാഗത്തിലേക്ക് പെടും.

**Bag of Words** (BoW) വെക്ടർ പ്രതിനിധാനം ഏറ്റവും സാധാരണമായി ഉപയോഗിക്കുന്ന പരമ്പരാഗത വെക്ടർ പ്രതിനിധാനമാണ്. ഓരോ വാക്കും ഒരു വെക്ടർ ഇൻഡക്സുമായി ബന്ധിപ്പിച്ചിരിക്കുന്നു, വെക്ടർ ഘടകം ഒരു നൽകിയ ഡോക്യുമെന്റിൽ ആ വാക്കിന്റെ ആവർത്തനങ്ങളുടെ എണ്ണം ഉൾക്കൊള്ളുന്നു.

![Image showing how a bag of words vector representation is represented in memory.](../../../../../translated_images/ml/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: BoW-നെ ടെക്സ്റ്റിലെ ഓരോ വാക്കിനും ഉള്ള ഒന്ന്-ഹോട്ട്-എൻകോഡഡ് വെക്ടറുകളുടെ മൊത്തം കൂട്ടമായി കാണാം.

താഴെ Scikit Learn പൈതൺ ലൈബ്രറി ഉപയോഗിച്ച് വാക്കുകളുടെ ബാഗ് പ്രതിനിധാനം എങ്ങനെ സൃഷ്ടിക്കാമെന്ന് ഒരു ഉദാഹരണം കൊടുത്തിരിക്കുന്നു:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

AG_NEWS ഡാറ്റാസെറ്റിന്റെ വെക്ടർ പ്രതിനിധാനത്തിൽ നിന്ന് ബാഗ്-ഓഫ്-വേർഡ്സ് വെക്ടർ കണക്കാക്കാൻ, നാം താഴെ കൊടുത്തിരിക്കുന്ന ഫംഗ്ഷൻ ഉപയോഗിക്കാം:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **കുറിപ്പ്:** ഇവിടെ നാം ഡിഫോൾട്ട് വാക്ക് സമാഹാരത്തിന്റെ വലിപ്പം നിർദ്ദേശിക്കാൻ ഗ്ലോബൽ `vocab_size` വേരിയബിൾ ഉപയോഗിക്കുന്നു. സാധാരണയായി വാക്ക് സമാഹാരത്തിന്റെ വലിപ്പം വളരെ വലുതായിരിക്കും, അതിനാൽ വാക്ക് സമാഹാരത്തിന്റെ വലിപ്പം ഏറ്റവും സാധാരണമായ വാക്കുകളിലേക്കു പരിമിതപ്പെടുത്താം. `vocab_size` മൂല്യം കുറച്ച് താഴെ കൊടുത്തിരിക്കുന്ന കോഡ് പ്രവർത്തിപ്പിച്ച് നോക്കൂ, അത് കൃത്യതയെ എങ്ങനെ ബാധിക്കുന്നുവെന്ന് കാണൂ. ചില കൃത്യത കുറവ് പ്രതീക്ഷിക്കാം, പക്ഷേ പ്രകടനത്തിൽ വലിയ വ്യത്യാസമുണ്ടാകില്ല.


## BoW ക്ലാസിഫയർ പരിശീലനം

നമ്മൾ ടെക്സ്റ്റിന്റെ Bag-of-Words പ്രതിനിധാനം എങ്ങനെ നിർമ്മിക്കാമെന്ന് പഠിച്ചപ്പോൾ, അതിന്റെ മുകളിൽ ഒരു ക്ലാസിഫയർ പരിശീലിപ്പിക്കാം. ആദ്യം, പരിശീലനത്തിനായി നമ്മുടെ ഡാറ്റാസെറ്റ് എല്ലാ സ്ഥാനവിവര വക്ടർ പ്രതിനിധാനങ്ങളും bag-of-words പ്രതിനിധാനങ്ങളായി മാറ്റേണ്ടതുണ്ട്. ഇത് സാധാരണ torch `DataLoader`-ലേക്ക് `collate_fn` പാരാമീറ്ററായി `bowify` ഫംഗ്ഷൻ നൽകിക്കൊണ്ട് സാധ്യമാക്കാം:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

ഇപ്പോൾ ഒരു ലീനിയർ ലെയർ അടങ്ങിയ ഒരു ലളിതമായ ക്ലാസിഫയർ ന്യൂറൽ നെറ്റ്‌വർക്ക് നിർവചിക്കാം. ഇൻപുട്ട് വെക്ടറിന്റെ വലുപ്പം `vocab_size`-നോട് തുല്യമാണ്, ഔട്ട്പുട്ട് വലുപ്പം ക്ലാസുകളുടെ എണ്ണം (4) അനുസരിച്ചാണ്. നാം ക്ലാസിഫിക്കേഷൻ ടാസ്ക് പരിഹരിക്കുന്നതിനാൽ, അവസാന ആക്ടിവേഷൻ ഫംഗ്ഷൻ `LogSoftmax()` ആണ്.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

ഇപ്പോൾ നാം സ്റ്റാൻഡേർഡ് PyTorch ട്രെയിനിംഗ് ലൂപ്പ് നിർവചിക്കും. നമ്മുടെ ഡാറ്റാസെറ്റ് വളരെ വലുതായതിനാൽ, പഠനത്തിനായി നാം ഒരു epoch മാത്രമേ ട്രെയിൻ ചെയ്യൂ, ചിലപ്പോൾ ഒരു epoch-ലും കുറവായി (epoch_size പാരാമീറ്റർ ഉപയോഗിച്ച് ട്രെയിനിംഗ് പരിധി നിശ്ചയിക്കാം). ട്രെയിനിംഗിനിടെ സമാഹരിച്ച accuracy-യും നാം റിപ്പോർട്ട് ചെയ്യും; റിപ്പോർട്ട് ചെയ്യാനുള്ള ആവൃത്തി report_freq പാരാമീറ്റർ ഉപയോഗിച്ച് നിശ്ചയിക്കാം.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## ബൈഗ്രാമുകൾ, ട്രൈഗ്രാമുകൾ, എൻ-ഗ്രാമുകൾ

ഒരു ബാഗ് ഓഫ് വേഡ്സ് സമീപനത്തിന്റെ ഒരു പരിമിതിയാണ് ചില വാക്കുകൾ മൾട്ടി വേഡ് എക്സ്പ്രഷനുകളുടെ ഭാഗമായിരിക്കുക എന്നത്. ഉദാഹരണത്തിന്, 'hot dog' എന്ന വാക്കിന് 'hot' ഉം 'dog' ഉം എന്ന വാക്കുകളുടെ വ്യത്യസ്തമായ അർത്ഥമുണ്ട്. 'hot' ഉം 'dog' ഉം എല്ലായ്പ്പോഴും ഒരേ വെക്ടറുകൾ ഉപയോഗിച്ച് പ്രതിനിധീകരിച്ചാൽ, അത് നമ്മുടെ മോഡലിനെ ആശയക്കുഴപ്പത്തിലാക്കാം.

ഇതിന് പരിഹാരമായി, **N-ഗ്രാം പ്രതിനിധാനങ്ങൾ** ഡോക്യുമെന്റ് ക്ലാസിഫിക്കേഷനിൽ ഉപയോഗിക്കാറുണ്ട്, ഇവിടെ ഓരോ വാക്കിന്റെയും, ബൈ-വേഡിന്റെയും, ട്രൈ-വേഡിന്റെയും ആവൃത്തി ക്ലാസിഫയർ പരിശീലനത്തിന് ഉപകാരപ്രദമായ ഫീച്ചറാണ്. ഉദാഹരണത്തിന്, ബൈഗ്രാം പ്രതിനിധാനത്തിൽ, നാം വാക്കുകളുടെ യോജിപ്പുകൾ എല്ലാം വാക്കുകളുടെ ശബ്ദകോശത്തിൽ ചേർക്കും, ഒറിജിനൽ വാക്കുകൾക്കൊപ്പം.

താഴെ Scikit Learn ഉപയോഗിച്ച് ബൈഗ്രാം ബാഗ് ഓഫ് വേഡ്സ് പ്രതിനിധാനം എങ്ങനെ സൃഷ്ടിക്കാമെന്ന് ഒരു ഉദാഹരണം കാണിക്കുന്നു:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-gram സമീപനത്തിന്റെ പ്രധാന ദോഷം വാക്കുകളുടെ വലുപ്പം അതിവേഗം വളരാൻ തുടങ്ങുന്നതാണ്. പ്രായോഗികമായി, N-gram പ്രതിനിധാനം ചില ഡൈമെൻഷണാലിറ്റി കുറയ്ക്കൽ സാങ്കേതികവിദ്യകളുമായി, ഉദാഹരണത്തിന് *എംബെഡിംഗുകൾ*, സംയോജിപ്പിക്കേണ്ടതുണ്ട്, ഇത് അടുത്ത യൂണിറ്റിൽ നാം ചർച്ച ചെയ്യും.

നമ്മുടെ **AG News** ഡാറ്റാസെറ്റിൽ N-gram പ്രതിനിധാനം ഉപയോഗിക്കാൻ, നാം പ്രത്യേക ngram വാക്ക് സമാഹാരം നിർമ്മിക്കേണ്ടതാണ്:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


അപ്പോൾ മുകളിൽ നൽകിയ കോഡ് തന്നെ ക്ലാസിഫയർ ട്രെയിൻ ചെയ്യാൻ ഉപയോഗിക്കാം, എന്നാൽ അത് മെമ്മറി കാര്യക്ഷമമല്ല. അടുത്ത യൂണിറ്റിൽ, നാം എംബെഡിംഗുകൾ ഉപയോഗിച്ച് ബിഗ്രാം ക്ലാസിഫയർ ട്രെയിൻ ചെയ്യും.

> **Note:** നിഗ്രാമുകൾ ടെക്സ്റ്റിൽ നിശ്ചിത തവണകളിൽ കൂടുതൽ സംഭവിക്കുന്നവ മാത്രമേ നിങ്ങൾ വയ്ക്കാൻ കഴിയൂ. ഇതിലൂടെ അപൂർവമായ ബിഗ്രാമുകൾ ഒഴിവാക്കപ്പെടുകയും, ഡൈമെൻഷണാലിറ്റി ഗണ്യമായി കുറയുകയും ചെയ്യും. ഇത് ചെയ്യാൻ, `min_freq` പാരാമീറ്റർ ഉയർന്ന മൂല്യത്തിൽ സജ്ജമാക്കി, വാക്കുകളുടെ വാക്ക്‌സഭയുടെ നീളം എങ്ങനെ മാറുന്നു എന്ന് ശ്രദ്ധിക്കുക.


## ടെർമ് ഫ്രീക്വൻസി ഇൻവേഴ്സ് ഡോക്യുമെന്റ് ഫ്രീക്വൻസി TF-IDF

BoW പ്രതിനിധാനത്തിൽ, വാക്കുകളുടെ സംഭവങ്ങൾ വാക്കിന്റെ സ്വഭാവം നോക്കാതെ സമാനമായി ഭാരം നൽകപ്പെടുന്നു. എന്നാൽ, *a*, *in* പോലുള്ള സാധാരണ വാക്കുകൾ ക്ലാസിഫിക്കേഷനിൽ പ്രത്യേകമായ വാക്കുകളേക്കാൾ വളരെ കുറവ് പ്രാധാന്യമുള്ളവയാണ് എന്ന് വ്യക്തമാകുന്നു. വാസ്തവത്തിൽ, പല NLP ജോലികളിലും ചില വാക്കുകൾ മറ്റുള്ളവയെക്കാൾ കൂടുതൽ പ്രസക്തമാണ്.

**TF-IDF** എന്നത് **term frequency–inverse document frequency** എന്നതിന്റെ ചുരുക്കമാണ്. ഇത് bag of words ന്റെ ഒരു വ്യത്യാസമാണ്, ഇവിടെ ഒരു വാക്ക് ഒരു ഡോക്യുമെന്റിൽ ഉണ്ടെന്നത് സൂചിപ്പിക്കുന്ന 0/1 ബൈനറി മൂല്യത്തിന് പകരം, കോർപ്പസിൽ വാക്കിന്റെ സംഭവങ്ങളുടെ ആവൃത്തി സംബന്ധിച്ച ഫ്ലോട്ടിംഗ് പോയിന്റ് മൂല്യം ഉപയോഗിക്കുന്നു.

കൂടുതൽ ഔപചാരികമായി, ഒരു വാക്ക് $i$ ഡോക്യുമെന്റ് $j$ ൽ ഉള്ള ഭാരം $w_{ij}$ ഇങ്ങനെ നിർവചിക്കുന്നു:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
ഇവിടെ
* $tf_{ij}$ എന്നത് $j$ ഡോക്യുമെന്റിൽ $i$ വാക്ക് എത്ര തവണ സംഭവിച്ചുവെന്ന്, അതായത് മുമ്പ് കണ്ട BoW മൂല്യം
* $N$ എന്നത് സമാഹാരത്തിലെ ഡോക്യുമെന്റുകളുടെ എണ്ണം
* $df_i$ എന്നത് ആ വാക്ക് $i$ ഉൾക്കൊള്ളുന്ന ഡോക്യുമെന്റുകളുടെ എണ്ണം മുഴുവൻ സമാഹാരത്തിൽ

TF-IDF മൂല്യം $w_{ij}$ ഒരു വാക്ക് ഒരു ഡോക്യുമെന്റിൽ എത്ര തവണ പ്രത്യക്ഷപ്പെടുന്നു എന്നതിന്റെ അനുപാതത്തിൽ വർദ്ധിക്കുന്നു, കൂടാതെ ആ വാക്ക് ഉൾക്കൊള്ളുന്ന ഡോക്യുമെന്റുകളുടെ എണ്ണം കൊണ്ട് കുറയ്ക്കപ്പെടുന്നു, ഇത് ചില വാക്കുകൾ മറ്റുള്ളവയെക്കാൾ കൂടുതലായി പ്രത്യക്ഷപ്പെടുന്ന കാര്യം പരിഗണിക്കാൻ സഹായിക്കുന്നു. ഉദാഹരണത്തിന്, ഒരു വാക്ക് സമാഹാരത്തിലെ *എല്ലാ* ഡോക്യുമെന്റുകളിലും ഉണ്ടെങ്കിൽ, $df_i=N$ ആകും, അതിനാൽ $w_{ij}=0$ ആകും, അത്തരത്തിലുള്ള വാക്കുകൾ പൂർണ്ണമായും അവഗണിക്കപ്പെടും.

Scikit Learn ഉപയോഗിച്ച് നിങ്ങൾക്ക് എളുപ്പത്തിൽ ടെക്സ്റ്റിന്റെ TF-IDF വെക്ടറൈസേഷൻ സൃഷ്ടിക്കാം:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## നിഗമനം

എങ്കിലും TF-IDF പ്രതിനിധാനങ്ങൾ വ്യത്യസ്ത വാക്കുകൾക്ക് ആവൃത്തി ഭാരമൊരുക്കിയിരുന്നാലും അവ അർത്ഥം അല്ലെങ്കിൽ ക്രമം പ്രതിനിധാനം ചെയ്യാൻ കഴിയുന്നില്ല. പ്രശസ്ത ഭാഷാശാസ്ത്രജ്ഞൻ J. R. Firth 1935-ൽ പറഞ്ഞതുപോലെ, "ഒരു വാക്കിന്റെ പൂർണ്ണ അർത്ഥം എപ്പോഴും സാന്ദർഭികമാണ്, സാന്ദർഭം ഒഴികെയുള്ള അർത്ഥ പഠനം ഗൗരവത്തോടെ സ്വീകരിക്കാനാകില്ല." ഈ കോഴ്സിൽ പിന്നീട് നാം ഭാഷാ മോഡലിംഗിലൂടെ ടെക്സ്റ്റിൽ നിന്നുള്ള സാന്ദർഭിക വിവരങ്ങൾ എങ്ങനെ പിടിച്ചെടുക്കാമെന്ന് പഠിക്കും.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖ അധികാരപരമായ ഉറവിടമായി കണക്കാക്കണം. നിർണായക വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനത്തിന്റെ ഉപയോഗത്തിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
